In [30]:
from dotenv import load_dotenv
import os
import textwrap
import json
import logging
from pathlib import Path
from typing import Dict, Any, List, Tuple

load_dotenv()

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from neo4j.exceptions import CypherSyntaxError

# Warning control
import warnings
warnings.filterwarnings("ignore")

## Create a graph from the RVT data

In [ ]:
path_to_graph_data = '../data/processed/rvt/case-autcon'

RELATION_LABELS: Dict[str, str] = {
    'e-adjacent_connectivity': 'ADJACENT',
    'e-spatial_containment': 'CONTAINED',
    'e-structural_support': 'SUPPORTS',
    'e-accessible_connectivity': 'ACCESSIBLE',
    'e-locational_alignment': 'ALIGNED',
}

def load_json_files(data_dir: str) -> Tuple[Dict[str, Dict[str, Any]], Dict[str, List[Tuple[str, str]]]]:
    """Load all vertex and edge JSON files from the data directory.
    Returns:
        - all_vertices: node_id -> properties (with 'node_type')
        - all_edges: relation_label -> list of (from_id, to_id)
    """
    data_path = Path(data_dir)
    
    # Find all JSON files
    vertex_files = sorted(data_path.glob('v-*.json'))
    edge_files = sorted(data_path.glob('e-*.json'))
    
    print(f"Found {len(vertex_files)} vertex files and {len(edge_files)} edge files")
    
    # Load all vertex files into a single dictionary
    all_vertices: Dict[str, Dict[str, Any]] = {}
    
    for v_file in vertex_files:
        vertex_type = v_file.stem[2:]  # Remove 'v-' prefix
        with open(v_file, 'r') as f:
            vertices = json.load(f)
            for node_id, properties in vertices.items():
                properties['node_type'] = vertex_type
                all_vertices[node_id] = properties
        
        print(f"  Loaded {len(vertices)} {vertex_type} nodes")
    
    # Load all edge files, grouping by semantic relation label inferred from filename
    all_edges: Dict[str, List[Tuple[str, str]]] = {}
    for e_file in edge_files:
        rel_label = RELATION_LABELS.get(e_file.stem)
        if rel_label is None:
            print(f"  Skipping unknown edge file type: {e_file.name}")
            continue
        
        with open(e_file, 'r') as f:
            edges_obj = json.load(f)
            # Each key inside is a node-type pair (e.g., 'space-stair'); we ignore the key
            # and collect all pairs under the semantic relation label.
            total_in_file = 0
            for _, edge_list in edges_obj.items():
                if rel_label not in all_edges:
                    all_edges[rel_label] = []
                all_edges[rel_label].extend((frm, to) for frm, to in edge_list)
                total_in_file += len(edge_list)
        
        print(f"  Loaded {total_in_file} '{rel_label}' edges from {e_file.name}")
    
    print(f"\nTotal nodes: {len(all_vertices)}")
    print(f"Total relation labels: {len(all_edges)}")
    print(f"Total relationships: {sum(len(edges) for edges in all_edges.values())}")
    
    return all_vertices, all_edges

# Load all data
all_vertices, all_edges = load_json_files(path_to_graph_data)


Found 8 vertex files and 5 edge files
  Loaded 120 column nodes
  Loaded 135 door nodes
  Loaded 19 separationline nodes
  Loaded 5 slab nodes
  Loaded 108 space nodes
  Loaded 7 stair nodes
  Loaded 191 wall nodes
  Loaded 87 window nodes
  Loaded 582 'ACCESSIBLE' edges from e-accessible_connectivity.json
  Loaded 1294 'ADJACENT' edges from e-adjacent_connectivity.json
  Loaded 673 'ALIGNED' edges from e-locational_alignment.json
  Loaded 289 'CONTAINED' edges from e-spatial_containment.json
  Loaded 1769 'SUPPORTS' edges from e-structural_support.json

Total nodes: 672
Total relation labels: 5
Total relationships: 4607


In [6]:
# Connect to Neo4j using Langchain Neo4jGraph
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI", "bolt://localhost:7687"),
    username=os.getenv("NEO4J_USER", "neo4j"),
    password=os.getenv("NEO4J_PASSWORD", 'password'),
    database="neo4j"
)

def clear_database(graph: Neo4jGraph) -> int:
    """Clear all nodes and relationships from the database."""
    graph.query("MATCH (n) DETACH DELETE n")
    result = graph.query("MATCH (n) RETURN count(n) as count")
    return result[0]["count"] if result else 0

def create_nodes_batch(graph: Neo4jGraph, nodes_batch: List[Tuple[str, Dict[str, Any]]]) -> int:
    """Create multiple nodes in a single batch."""
    created = 0
    
    for node_id, properties in nodes_batch:
        node_type = properties.get('node_type', 'Entity')
        label = node_type.capitalize()
        
        # Build properties for the query
        props = {k: v for k, v in properties.items() if k != 'node_type'}
        props['id'] = node_id
        
        # Create Cypher query with parameters (label must be in query string, not parameterized)
        query = f"CREATE (n:{label} $props)"
        
        # Execute query with parameters
        graph.query(query, params={"props": props})
        created += 1
    
    return created

def create_relationships_batch(graph: Neo4jGraph, edges_batch: List[Tuple[str, str, str]]) -> int:
    """Create multiple relationships in a single batch."""
    created = 0
    
    for from_id, to_id, edge_type in edges_batch:
        rel_label = edge_type.upper().replace('-', '_')
        
        # Relationship type must be in query string, not parameterized
        query = f"""
        MATCH (a) WHERE a.id = $from_id
        MATCH (b) WHERE b.id = $to_id
        CREATE (a)-[r:{rel_label}]->(b)
        """
        
        graph.query(query, params={"from_id": from_id, "to_id": to_id})
        created += 1
    
    return created


# Process nodes
print("Creating nodes...")
# Clear existing data
print("Clearing existing database...")
count = clear_database(graph)
print(f"Deleted {count} existing nodes")

# Create all nodes in batches
batch_size = 100
node_count = 0
nodes_list = list(all_vertices.items())

for i in range(0, len(nodes_list), batch_size):
    batch = nodes_list[i:i+batch_size]
    create_nodes_batch(graph, batch)
    node_count += len(batch)
    if node_count % 500 == 0:
        print(f"  Created {node_count}/{len(nodes_list)} nodes...")

print(f"Created {node_count} nodes")

# Create all relationships in batches
print("\nCreating relationships...")
rel_count = 0
edges_list = []

for edge_type, edges in all_edges.items():
    print(f"  Preparing {edge_type} relationships ({len(edges)} edges)...")
    for from_id, to_id in edges:
        edges_list.append((from_id, to_id, edge_type))

print(f"Creating {len(edges_list)} total relationships in batches...")
for i in range(0, len(edges_list), batch_size):
    batch = edges_list[i:i+batch_size]
    create_relationships_batch(graph, batch)
    rel_count += len(batch)
    if rel_count % 500 == 0:
        print(f"  Created {rel_count}/{len(edges_list)} relationships...")

print(f"Created {rel_count} relationships")
print("\nDone! Data successfully loaded into Neo4j.")

Creating nodes...
Clearing existing database...
Deleted 0 existing nodes
  Created 500/672 nodes...
Created 672 nodes

Creating relationships...
  Preparing ACCESSIBLE relationships (582 edges)...
  Preparing ADJACENT relationships (1294 edges)...
  Preparing ALIGNED relationships (673 edges)...
  Preparing CONTAINED relationships (289 edges)...
  Preparing SUPPORTS relationships (1769 edges)...
Creating 4607 total relationships in batches...
  Created 500/4607 relationships...
  Created 1000/4607 relationships...
  Created 1500/4607 relationships...
  Created 2000/4607 relationships...
  Created 2500/4607 relationships...
  Created 3000/4607 relationships...
  Created 3500/4607 relationships...
  Created 4000/4607 relationships...
  Created 4500/4607 relationships...
Created 4607 relationships

Done! Data successfully loaded into Neo4j.


In [7]:
from typing import List

# Verify the data was loaded successfully using Neo4jGraph
def get_stats(graph: Neo4jGraph) -> tuple:
    """Get statistics about the loaded graph."""
    # Count nodes by label
    node_counts = graph.query("""
        MATCH (n)
        WITH labels(n) as labels
        UNWIND labels as label
        RETURN label, count(*) as count
        ORDER BY count DESC
    """)
    
    # Count relationships by type
    rel_counts = graph.query("""
        MATCH ()-[r]->()
        RETURN type(r) as rel_type, count(*) as count
        ORDER BY count DESC
    """)
    
    return node_counts, rel_counts

def get_sample_relationships(graph: Neo4jGraph, rel_type: str, limit: int = 5) -> List:
    """Get sample relationships of a given type."""
    query = f"""
    MATCH (a)-[r:{rel_type}]->(b)
    RETURN a.id as from_id, b.id as to_id, 
           labels(a)[0] as from_type, labels(b)[0] as to_type
    LIMIT $limit
    """
    return graph.query(query, params={"limit": limit})

# Get node statistics
node_counts, rel_counts = get_stats(graph)

print("=== NODE STATISTICS ===")
total_nodes = sum(row["count"] for row in node_counts)
print(f"Total nodes: {total_nodes}")
for row in node_counts:
    print(f"  {row['label']}: {row['count']}")

print("\n=== RELATIONSHIP STATISTICS ===")
total_rels = sum(row["count"] for row in rel_counts)
print(f"Total relationships: {total_rels}")
for row in rel_counts:
    print(f"  {row['rel_type']}: {row['count']}")

# Show sample relationships
if rel_counts:
    first_rel_type = rel_counts[0]["rel_type"]
    print(f"\n=== SAMPLE RELATIONSHIPS (type: {first_rel_type}) ===")
    samples = get_sample_relationships(graph, first_rel_type, 5)
    for row in samples:
        print(f"  {row['from_type']}({row['from_id'][:10]}...) -> {row['to_type']}({row['to_id'][:10]}...)")

=== NODE STATISTICS ===
Total nodes: 672
  Wall: 191
  Door: 135
  Column: 120
  Space: 108
  Window: 87
  Separationline: 19
  Stair: 7
  Slab: 5

=== RELATIONSHIP STATISTICS ===
Total relationships: 4607
  SUPPORTS: 1769
  ADJACENT: 1294
  ALIGNED: 673
  ACCESSIBLE: 582
  CONTAINED: 289

=== SAMPLE RELATIONSHIPS (type: SUPPORTS) ===
  Column(2683100...) -> Column(2683143...)
  Column(2683101...) -> Column(2683144...)
  Column(2683103...) -> Column(2683146...)
  Column(2610245...) -> Column(2683119...)
  Column(2572075...) -> Column(2601466...)


## Query the graph

In [2]:
# Connect to Neo4j using Langchain Neo4jGraph
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI", "bolt://localhost:7687"),
    username=os.getenv("NEO4J_USER", "neo4j"),
    password=os.getenv("NEO4J_PASSWORD", 'password'),
    database="neo4j"
)

In [3]:
schema_json = graph.query("""CALL apoc.meta.schema() YIELD value
UNWIND keys(value) AS name
WITH name, value[name] AS data
WHERE data.type IN ['node', 'relationship']
RETURN 
    name, 
    data.type AS type, 
    keys(data.properties) AS properties
ORDER BY type, name
""")

schema_json


[{'name': 'Column',
  'type': 'node',
  'properties': ['ifc_guid', 'location', 'id', 'level_id']},
 {'name': 'Door',
  'type': 'node',
  'properties': ['ifc_guid', 'location', 'id', 'level_id']},
 {'name': 'Separationline',
  'type': 'node',
  'properties': ['ifc_guid', 'location', 'id', 'level_id']},
 {'name': 'Slab',
  'type': 'node',
  'properties': ['ifc_guid', 'location', 'id', 'level_id']},
 {'name': 'Space',
  'type': 'node',
  'properties': ['ifc_guid',
   'bound_phy',
   'number',
   'bound_vrt',
   'level_id',
   'width',
   'length',
   'name',
   'location',
   'id']},
 {'name': 'Stair',
  'type': 'node',
  'properties': ['ifc_guid',
   'top_elevation',
   'location',
   'id',
   'base_elevation',
   'horizontal_dimensions']},
 {'name': 'Wall',
  'type': 'node',
  'properties': ['ifc_guid',
   'location',
   'is_room_bounding',
   'id',
   'level_id']},
 {'name': 'Window',
  'type': 'node',
  'properties': ['ifc_guid', 'location', 'id', 'level_id']},
 {'name': 'ACCESSIBLE',

In [4]:
all_nodes_query = """MATCH (n)
    WHERE size(labels(n)) > 0 
    WITH labels(n)[0] AS label, collect(n) AS nodes
    RETURN label, nodes[0] AS sampleNode"""

all_nodes =graph.query(all_nodes_query)

all_nodes

[{'label': 'Column',
  'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjNmt',
   'level_id': '79627',
   'location': [96.06087148282784, 102.24828131414463, 24.27821522309711],
   'id': '2683100'}},
 {'label': 'Door',
  'sampleNode': {'ifc_guid': '2P0Clh2c952RPyuPFEYgcQ',
   'level_id': '24770',
   'location': [28.524353627965954, 43.365276144551295, 0.0],
   'id': '2642808'}},
 {'label': 'Separationline',
  'sampleNode': {'ifc_guid': '1yPJv2Vxf8r9twVNmFuZ2v',
   'level_id': '2680262',
   'location': [39.08435362796594,
    27.16527614455126,
    11.1,
    39.08435362796593,
    21.56527614455119,
    11.1],
   'id': '2779188'}},
 {'label': 'Slab',
  'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjMhM',
   'level_id': '2680262',
   'location': [34.269353627965906, 27.47527614455132, 10.950000000000003],
   'id': '2680381'}},
 {'label': 'Space',
  'sampleNode': {'ifc_guid': '3RD7PAHIz7R9lDzWO$cjQK',
   'bound_phy': ['2670852', '2670853', '2534518', '2535474'],
   'number': '110',
   'bound

In [5]:
issues = '/Users/stefanfuchs/Repos/acc-recomm/data/processed/rule/case-autcon/issues.json'

with open(issues, 'r') as f:
    issues = json.load(f)

issues[:2]

[{'regulation_clause': '1007_1_1',
  'core_component_type': 'Conference Room',
  'core_component_type_number': '306',
  'core_component_GUID': '1W6yAWYtbDbPaMjexfopdL',
  'checking_variable_name': 'Required door separation',
  'checking_variable_unit': 'feet',
  'required_variable_value': '20.82',
  'actual_variable_value': '17.37',
  'bcf_id': 'df6a3d56-8685-4675-b5a2-7650e85088db'},
 {'regulation_clause': '1007_1_1',
  'core_component_type': 'Conference Room',
  'core_component_type_number': '307',
  'core_component_GUID': '1W6yAWYtbDbPaMjexfopdJ',
  'checking_variable_name': 'Required door separation',
  'checking_variable_unit': 'feet',
  'required_variable_value': '24.51',
  'actual_variable_value': '17.40',
  'bcf_id': '21a38776-fb87-4b71-b5c2-e92e23f2182a'}]

In [43]:
pwd

'/Users/stefanfuchs/Repos/acc-recomm/notebooks'

In [46]:
import json
issues = '../data/processed/acc_result/case-autcon/issues/topics.json'

with open(issues, 'r') as f:
    issues = json.load(f)

len(issues)

56

In [6]:
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
OPENROUTER_BASE_URL = os.getenv('OPENROUTER_BASE_URL', 'https://openrouter.ai/api/v1')

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Be specific with your Cypher statements. You can make it less restrictive in a second step if no results are found.

For example, if one wants to retrieve examples for all the nodes, one could use the following Cypher statement:
- Example query: {all_nodes_query}
- Example result: {all_nodes}


The question is:
{question}"""

In [31]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question", "all_nodes_query", "all_nodes"],
    template=CYPHER_GENERATION_TEMPLATE
)

llm = ChatOpenAI(temperature=0, openai_api_key=OPENROUTER_API_KEY, model="google/gemini-2.5-flash", base_url=OPENROUTER_BASE_URL)

cypherChain = GraphCypherQAChain.from_llm(
    llm,
    allow_dangerous_requests=True,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

logger = logging.getLogger(__name__)

def run_graph_cypher_tool(query: str) -> Dict[str, Any]:
    payload = {
        "query": query,
        "all_nodes_query": all_nodes_query,
        "all_nodes": str(all_nodes),
    }
    try:
        return cypherChain.invoke(payload)
    except CypherSyntaxError as exc:
        logger.warning("Cypher syntax error when executing generated query: %s", exc)
        return {
            "error": "CypherSyntaxError",
            "message": str(exc),
            "payload": payload,
        }

def prettyCypherChain(question: str) -> Dict[str, Any]:
    response = run_graph_cypher_tool(question)
    print(textwrap.fill(str(response), 160))
    return response

In [8]:
prettyCypherChain(f"I've got the following issue: {issues[1]}. How could I fix it?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Space {id: '307'})-[:ACCESSIBLE]->(d:Door)
WHERE d.location[0] < 24.51
RETURN s, d

Full Context:
[]

> Finished chain.
{'query': "I've got the following issue: {'regulation_clause': '1007_1_1', 'core_component_type': 'Conference Room', 'core_component_type_number': '307',
'core_component_GUID': '1W6yAWYtbDbPaMjexfopdJ', 'checking_variable_name': 'Required door separation', 'checking_variable_unit': 'feet',
'required_variable_value': '24.51', 'actual_variable_value': '17.40', 'bcf_id': '21a38776-fb87-4b71-b5c2-e92e23f2182a'}. How could I fix it?",
'all_nodes_query': 'MATCH (n)\n    WHERE size(labels(n)) > 0 \n    WITH labels(n)[0] AS label, collect(n) AS nodes\n    RETURN label, nodes[0] AS sampleNode',
'all_nodes': "[{'label': 'Column', 'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjNmt', 'level_id': '79627', 'location': [96.06087148282784, 102.24828131414463,
24.27821522309711], 'id': '2683100'}}, {'label'

In [32]:
from langchain.agents import Tool, initialize_agent, AgentType

clause_text = 'Where two exits, exit access doorways, exit access stairways or ramps, or any combination thereof, are required from any portion of the exit access, they shall be placed a distance apart equal to not less than one-half of the length of the maximum overall diagonal dimension of the building or area to be served measured in a straight line between them. Interlocking or scissor stairways shall be counted as one exit stairway.'

tool = Tool(
    name="graph-cypher-qa",
    func=run_graph_cypher_tool,
    description="Answer questions about the building graph using Cypher."
)

agent = initialize_agent(
    tools=[tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,
    early_stopping_method="generate"
)

result = agent.run(f"""I've got the following issue: 
<start_regulation>{clause_text}</start_regulation>
<issue>{issues[1]}</issue> 

How could I fix it? Provide a high-level description of what elements of the building are involved and what type of action is needed. 
Note: Ignore the naming convention of the issue, just focus on the graph schema when writing the Cypher query.""")
print(result)



> Entering new AgentExecutor chain...
The user is asking for a high-level description of how to fix an issue related to exit separation. I need to identify the building elements involved and the type of action needed based on the provided regulation and issue details.

The issue states that the 'actual_variable_value' (17.40 feet) for 'Required door separation' is less than the 'required_variable_value' (24.51 feet) for a 'Conference Room' (core_component_type). The regulation specifies that "Where two exits... are required... they shall be placed a distance apart equal to not less than one-half of the length of the maximum overall diagonal dimension of the building or area to be served".

This means the exits for Conference Room 307 are too close together. To fix this, the exits need to be moved further apart.

I will use the `graph-cypher-qa` tool to get more information about the 'Conference Room' and its associated 'Exits' to provide a more detailed answer. I will look for the 'C

Cypher syntax error when executing generated query: {neo4j_code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'There': expected 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'FOREACH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"There are no ConferenceRoom or Exit nodes, nor are there HAS_EXIT relationships in the schema."
 ^} {gql_status: 42001} {gql_status_description: error: syntax error or access rule violation - invalid syntax}


Generated Cypher:
There are no ConferenceRoom or Exit nodes, nor are there HAS_EXIT relationships in the schema.

Observation: {'error': 'CypherSyntaxError', 'message': '{neo4j_code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input \'There\': expected \'ALTER\', \'ORDER BY\', \'CALL\', \'USING PERIODIC COMMIT\', \'CREATE\', \'LOAD CSV\', \'START DATABASE\', \'STOP DATABASE\', \'DEALLOCATE\', \'DELETE\', \'DENY\', \'DETACH\', \'DROP\', \'DRYRUN\', \'FINISH\', \'FOREACH\', \'GRANT\', \'INSERT\', \'LIMIT\', \'MATCH\', \'MERGE\', \'NODETACH\', \'OFFSET\', \'OPTIONAL\', \'REALLOCATE\', \'REMOVE\', \'RENAME\', \'RETURN\', \'REVOKE\', \'ENABLE SERVER\', \'SET\', \'SHOW\', \'SKIP\', \'TERMINATE\', \'UNWIND\', \'USE\' or \'WITH\' (line 1, column 1 (offset: 0))\n"There are no ConferenceRoom or Exit nodes, nor are there HAS_EXIT relationships in the schema."\n ^} {gql_status: 42001} {gql_status_description: error: syntax error or access rule violation - invalid syntax}', 'payload': 

In [10]:
result = agent.run(f"""I've got the following issue: {issues[-1]}. How could I fix it? Provide a high-level description of what elements of the building are involved and what type of action is needed. 
        Note: Ignore the naming convention of the issue, just focus on the graph schema when writing the Cypher query.""")
print(result)



> Entering new AgentExecutor chain...
To address the issue, I need to identify the relevant elements in the building graph that correspond to the provided details. The issue involves a corridor component with a specific width requirement that is not being met. I will look for the corridor component in the graph and check its properties, particularly focusing on the width. 

Action: graph-cypher-qa  
Action Input: "MATCH (c:CoreComponent {type: 'Corridor', GUID: '3O7NDSvVP7jhJX5Gt$LDET'}) RETURN c"  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c) WHERE size(labels(c)) > 0 WITH labels(c)[0] AS label, collect(c) AS nodes RETURN label, nodes[0] AS sampleNode
Full Context:
[{'label': 'Column', 'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjNmt', 'level_id': '79627', 'location': [96.06087148282784, 102.24828131414463, 24.27821522309711], 'id': '2683100'}}, {'label': 'Door', 'sampleNode': {'ifc_guid': '2P0Clh2c952RPyuPFEYgcQ', 'level_id': '24770', 'location': [28.52

In [20]:
# Structured AdaptationPlan with full objects per suggestion
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field, field_validator
import json

class Suggestion(BaseModel):
    # Store objects as JSON string to avoid Dict[str, Any] schema issues with OpenAI
    objects_json: str = Field(
        default="[]",
        description="JSON string of full objects referenced by the suggestion (e.g., Space, Door, Wall).",
    )
    ids: Optional[List[str]] = Field(
        default=None,
        description="Optional ids extracted from the objects or context.",
    )
    action: str = Field(description="Concise action proposal")
    reasoning: str = Field(description="Brief reasoning for the action")
    
    @property
    def objects(self) -> List[Dict[str, Any]]:
        """Parse objects_json to get the list of objects."""
        return json.loads(self.objects_json) if self.objects_json else []
    
    @field_validator('objects_json')
    @classmethod
    def validate_objects_json(cls, v: str) -> str:
        """Ensure objects_json is valid JSON."""
        if v:
            json.loads(v)  # Validate JSON
        return v

class AdaptationPlan(BaseModel):
    suggestions: List[Suggestion] = Field(
        description="Exactly three adaptation suggestions",
        min_items=3,
        max_items=3,
        default_factory=list,
    )

structured_llm_objects = llm.with_structured_output(AdaptationPlan)

agent = initialize_agent(
    tools=[tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,
    early_stopping_method="generate",
    handle_parsing_errors=True,
)

agent_prompt_objects = f"""I've got the following issue: 
<start_regulation>{clause_text}</start_regulation>
<issue>{issues[1]}</issue>

Objectives:
1. Use the `graph-cypher-qa` tool to fetch the actual node objects (Space, Door, Wall, Stair, Corridor, etc.) involved in the issue.
2. Preserve the full JSON dictionaries returned by the tool. Do not summarize or reformat them.
3. Produce exactly three adaptation suggestions grounded in those nodes.

Final response format (strict):
Final Answer:
{{
  "suggestions": [
    {{
      "action": "...",
      "reasoning": "...",
      "objects_json": "[{{\"id\": \"2512768\", \"ifc_guid\": \"1WNDqqV1rAAAY3T7Oc5Qcy\", \"is_room_bounding\": true, \"level_id\": \"24770\", \"location\": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0]}}]",
      "ids": ["..."]
    }},
    ... two more suggestion objects ...
  ]
}}

Rules:
- Replace the ellipses with real content.
- `objects_json` must be a JSON array string containing the exact node dictionaries you obtained (copy/paste the dicts and ensure the string is valid JSON).
- `ids` must list every `id` present in `objects_json` (empty array if none).
- Do not output any text before `Final Answer:` or after the JSON object.
"""

agent_answer_objects: str = agent.run(agent_prompt_objects)

# Example wall object format
wall_example = {
    "ifc_guid": "1WNDqqV1rAAAY3T7Oc5Qcy",
    "is_room_bounding": True,
    "level_id": "24770",
    "location": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0],
    "id": "2512768"
}

example_objects_json = json.dumps([wall_example], ensure_ascii=False)

final_plan_objects: AdaptationPlan = structured_llm_objects.invoke(
    f"""Issue: {json.dumps(issues[1], ensure_ascii=False)}
    AgentAnswer: {agent_answer_objects}
    Produce exactly three suggestions. For each suggestion:
    - Include full JSON objects as a JSON array string in 'objects_json' field.
    - Example format for objects_json (Wall object): '{example_objects_json}'
    - If an 'id' field is present in those objects, extract and set 'ids' to a list of those id values; otherwise leave ids as null.
    - The objects should be the actual node objects from the graph (Space, Door, Wall, etc.) that are relevant to the suggestion."""
)

print(final_plan_objects.model_dump_json(indent=2))




> Entering new AgentExecutor chain...
Thought: I need to fetch the actual node objects related to the issue, specifically focusing on the Conference Room and its associated components like doors, walls, and exits. I will use the `graph-cypher-qa` tool to retrieve this information.

Action: graph-cypher-qa
Action Input: MATCH (n) WHERE n.core_component_type = 'Conference Room' OR n.core_component_GUID = '1W6yAWYtbDbPaMjexfopdJ' RETURN n

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n) WHERE n.id = 'Conference Room' OR n.ifc_guid = '1W6yAWYtbDbPaMjexfopdJ' RETURN n
Full Context:
[{'n': {'ifc_guid': '1W6yAWYtbDbPaMjexfopdJ', 'bound_phy': ['2682946', '2682934', '2682935', '2682933'], 'number': '307', 'bound_vrt': [], 'level_id': '2680262', 'width': 4.8000000000000265, 'length': 14.150000000000007, 'name': 'Conference Room', 'location': [34.086932613103635, 34.967800108295116, 11.1], 'id': '2686161'}}]

> Finished chain.

Observation: {'query': "MATCH (n) WHERE n.co

In [22]:
structured_llm_objects = llm.with_structured_output(AdaptationPlan)

agent = initialize_agent(
    tools=[tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,
    early_stopping_method="generate",
    handle_parsing_errors=True,
)

agent_prompt_objects = f"""I've got the following issue: 
<start_regulation>{clause_text}</start_regulation>
<issue>{issues[1]}</issue>

Objectives:
1. Use the `graph-cypher-qa` tool to fetch the actual node objects (Space, Door, Wall, Stair, Corridor, etc.) involved in the issue.
2. Preserve the full JSON dictionaries returned by the tool. Do not summarize or reformat them.
3. Produce exactly three adaptation suggestions grounded in those nodes.

Final response format (strict):
Final Answer:
{{
  "suggestions": [
    {{
      "action": "...",
      "reasoning": "...",
      "objects_json": "[{{\"id\": \"2512768\", \"ifc_guid\": \"1WNDqqV1rAAAY3T7Oc5Qcy\", \"is_room_bounding\": true, \"level_id\": \"24770\", \"location\": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0]}}]",
      "ids": ["..."]
    }},
    ... two more suggestion objects ...
  ]
}}

Rules:
- Replace the ellipses with real content.
- `objects_json` must be a JSON array string containing the exact node dictionaries you obtained (copy/paste the dicts and ensure the string is valid JSON).
- `ids` must list every `id` present in `objects_json` (empty array if none).
- Do not output any text before `Final Answer:` or after the JSON object.
"""

agent_answer_objects: str = agent.run(agent_prompt_objects)

# Example wall object format
wall_example = {
    "ifc_guid": "1WNDqqV1rAAAY3T7Oc5Qcy",
    "is_room_bounding": True,
    "level_id": "24770",
    "location": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0],
    "id": "2512768"
}

example_objects_json = json.dumps([wall_example], ensure_ascii=False)

final_plan_objects: AdaptationPlan = structured_llm_objects.invoke(
    f"""Issue: {json.dumps(issues[1], ensure_ascii=False)}
    AgentAnswer: {agent_answer_objects}
    Produce exactly three suggestions. For each suggestion:
    - Include full JSON objects as a JSON array string in 'objects_json' field.
    - Example format for objects_json (Wall object): '{example_objects_json}'
    - If an 'id' field is present in those objects, extract and set 'ids' to a list of those id values; otherwise leave ids as null.
    - The objects should be the actual node objects from the graph (Space, Door, Wall, etc.) that are relevant to the suggestion."""
)

print(final_plan_objects.model_dump_json(indent=2))




> Entering new AgentExecutor chain...
Thought: The user wants to get adaptation suggestions based on a given regulation and issue.
I need to use the `graph-cypher-qa` tool to fetch the relevant node objects.
The issue specifies `core_component_type`: 'Conference Room', `core_component_type_number`: '307', and `core_component_GUID`: '1W6yAWYtbDbPaMjexfopdJ'. This is likely the main component involved.
The regulation talks about "two exits, exit access doorways, exit access stairways or ramps". The issue is about "Required door separation" and the `actual_variable_value` is less than the `required_variable_value`. This suggests that the doors serving the Conference Room 307 are too close.

I should query for the Conference Room 307 and its associated doors.
Action: graph-cypher-qa
Action Input: What are the doors associated with Conference Room 307 with GUID '1W6yAWYtbDbPaMjexfopdJ'?

> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Space {name: 'Conference 

In [23]:
# Structured AdaptationPlan with full objects per suggestion
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field, field_validator
import json

class Suggestion(BaseModel):
    # Store objects as JSON string to avoid Dict[str, Any] schema issues with OpenAI
    objects_json: str = Field(
        default="[]",
        description="JSON string of full objects referenced by the suggestion (e.g., Space, Door, Wall).",
    )
    context_json: str = Field(
        default="[]",
        description="JSON string of context objects referenced by the suggestion (e.g., a door is moved within it's wall, and the wall is moved in the context of a space).",
    )
    ids: Optional[List[str]] = Field(
        default=None,
        description="Optional ids extracted from the objects or context.",
    )
    action: str = Field(description="Concise action proposal")
    reasoning: str = Field(description="Brief reasoning for the action")
    
    @property
    def objects(self) -> List[Dict[str, Any]]:
        """Parse objects_json to get the list of objects."""
        return json.loads(self.objects_json) if self.objects_json else []
    
    @field_validator('objects_json', 'context_json')
    @classmethod
    def validate_objects_json(cls, v: str) -> str:
        """Ensure objects_json is valid JSON."""
        if v:
            json.loads(v)  # Validate JSON
        return v

class AdaptationPlan(BaseModel):
    suggestions: List[Suggestion] = Field(
        description="Exactly three adaptation suggestions",
        min_items=3,
        max_items=3,
        default_factory=list,
    )

structured_llm_objects = llm.with_structured_output(AdaptationPlan)

agent = initialize_agent(
    tools=[tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,
    early_stopping_method="generate",
    handle_parsing_errors=True,
)

agent_prompt_objects = f"""I've got the following issue: 
<start_regulation>{clause_text}</start_regulation>
<issue>{issues[1]}</issue>

Objectives:
1. Use the `graph-cypher-qa` tool to fetch the actual node objects (Space, Door, Wall, Stair, Corridor, etc.) involved in the issue.
2. Preserve the full JSON dictionaries returned by the tool. Do not summarize or reformat them.
3. Produce exactly three adaptation suggestions grounded in those nodes.

Final response format (strict):
Final Answer:
{{
  "suggestions": [
    {{
      "action": "...",
      "reasoning": "...",
      "objects_json": "[{{\"id\": \"2512768\", \"ifc_guid\": \"1WNDqqV1rAAAY3T7Oc5Qcy\", \"is_room_bounding\": true, \"level_id\": \"24770\", \"location\": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0]}}]",
      "context_json": "[{{\"ifc_guid\": \"1yPJv2Vxf8r9twVNmFuZ2v\", \"number\": \"110\", \"level_id\": \"25474\", \"name\": \"Office\", \"location\": [40.4781958073382, 8.812300113685527, 3.7], \"id\": \"2655881\"}}]",
      "ids": ["..."]
    }},
    ... two more suggestion objects ...
  ]
}}

Rules:
- Replace the ellipses with real content.
- `objects_json` must be a JSON array string containing the exact node dictionaries you obtained (copy/paste the dicts and ensure the string is valid JSON).
- `ids` must list every `id` present in `objects_json` (empty array if none).
- Do not output any text before `Final Answer:` or after the JSON object.
"""

agent_answer_objects: str = agent.run(agent_prompt_objects)

# Example wall object format
wall_example = {
    "ifc_guid": "1WNDqqV1rAAAY3T7Oc5Qcy",
    "is_room_bounding": True,
    "level_id": "24770",
    "location": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0],
    "id": "2512768"
}

example_objects_json = json.dumps([wall_example], ensure_ascii=False)

final_plan_objects: AdaptationPlan = structured_llm_objects.invoke(
    f"""Issue: {json.dumps(issues[1], ensure_ascii=False)}
    AgentAnswer: {agent_answer_objects}
    Produce exactly three suggestions. For each suggestion:
    - Include full JSON objects as a JSON array string in 'objects_json' field.
    - Example format for objects_json (Wall object): '{example_objects_json}'
    - If an 'id' field is present in those objects, extract and set 'ids' to a list of those id values; otherwise leave ids as null.
    - The objects should be the actual node objects from the graph (Space, Door, Wall, etc.) that are relevant to the suggestion."""
)

print(final_plan_objects.model_dump_json(indent=2))




> Entering new AgentExecutor chain...
Thought: The user wants to get adaptation suggestions for a given issue.
First, I need to identify the core component involved in the issue, which is a 'Conference Room' with GUID '1W6yAWYtbDbPaMjexfopdJ'.
The issue is about 'Required door separation' not meeting the 'required_variable_value'. This means the doors are too close.
I need to find the doors associated with this Conference Room and their properties.
Then, I will formulate three suggestions based on moving or adding doors to meet the separation requirement.
I will use the `graph-cypher-qa` tool to fetch the relevant nodes.

Action: graph-cypher-qa
Action Input: MATCH (c:Space {ifc_guid: '1W6yAWYtbDbPaMjexfopdJ'})-[r:HAS_DOOR]->(d:Door) RETURN c, COLLECT(d) AS doors

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Space {ifc_guid: '1W6yAWYtbDbPaMjexfopdJ'})-[:ACCESSIBLE]->(d:Door) RETURN c, COLLECT(d) AS doors
Full Context:
[{'c': {'ifc_guid': '1W6yAWYtbDbPaMjexfop

In [12]:
issues[:20]

[{'regulation_clause': '1007_1_1',
  'core_component_type': 'Conference Room',
  'core_component_type_number': '306',
  'core_component_GUID': '1W6yAWYtbDbPaMjexfopdL',
  'checking_variable_name': 'Required door separation',
  'checking_variable_unit': 'feet',
  'required_variable_value': '20.82',
  'actual_variable_value': '17.37',
  'bcf_id': 'df6a3d56-8685-4675-b5a2-7650e85088db'},
 {'regulation_clause': '1007_1_1',
  'core_component_type': 'Conference Room',
  'core_component_type_number': '307',
  'core_component_GUID': '1W6yAWYtbDbPaMjexfopdJ',
  'checking_variable_name': 'Required door separation',
  'checking_variable_unit': 'feet',
  'required_variable_value': '24.51',
  'actual_variable_value': '17.40',
  'bcf_id': '21a38776-fb87-4b71-b5c2-e92e23f2182a'},
 {'regulation_clause': '1017_2',
  'core_component_type': 'Conference Room',
  'core_component_type_number': '307',
  'core_component_GUID': '1W6yAWYtbDbPaMjexfopdJ',
  'checking_variable_name': 'Travel distance to safe plac

In [14]:
from langchain.agents import Tool, initialize_agent, AgentType
from pydantic import BaseModel, Field
from typing import List

# Tool returns full chain response so the agent can see intermediate steps (ids in context)
structured_tool = Tool(
    name="graph-cypher-qa",
    func=run_graph_cypher_tool,
    description="Answer questions about the building graph using Cypher.")

agent = initialize_agent(
    tools=[structured_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,
    early_stopping_method="generate",
)

class Suggestion(BaseModel):
    ids: List[str] = Field(default_factory=list, description="Relevant node ids (e.g., spaces, doors, walls)")
    action: str = Field(description="Concise action proposal")
    reasoning: str = Field(description="Brief reasoning for the action")

class AdaptationPlan(BaseModel):
    suggestions: List[Suggestion] = Field(
        description="Exactly three adaptation suggestions",
        min_items=3,
        max_items=3,
        default_factory=list,
    )

structured_llm = llm.with_structured_output(AdaptationPlan)

agent_prompt = f"""I've got the following issue: {issues[1]}.
How could I fix it? Provide three distinct adaptation suggestions.
- Use at most 2 tool calls.
- Ensure your Cypher returns relevant node ids when possible (e.g., space.id, door.id).
"""

agent_answer: str = agent.run(agent_prompt)

# Ask the structured model to summarize into the schema; include the issue and the agent answer
final_plan: AdaptationPlan = structured_llm.invoke(
    f"Issue: {json.dumps(issues[1], ensure_ascii=False)}\n"
    f"AgentAnswer: {agent_answer}\n"
    "Produce exactly three suggestions. Extract node ids explicitly mentioned; if none, leave ids empty."
)

print(final_plan.model_dump_json(indent=2))




> Entering new AgentExecutor chain...
To address the issue regarding the required door separation in the conference room, I need to gather information about the existing door configurations and potential solutions. I will first look for the door associated with the conference room to understand its current specifications and then explore possible adaptations.

Action: graph-cypher-qa  
Action Input: "MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:HAS_DOOR]->(d:Door) RETURN c.id, d.id, d.separation"

> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `GUID` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=1, column=25, offset=24>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 24, 'line': 1, 'column': 25}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:ACCESSIBLE]->(d:Door) RETURN c.id, d.id"
Received notification from DBMS server: <GqlStatusObject gql_status='01N50', status_description='warn: label does not exist. The label `CoreComponent` does not exist in database `neo4j`. Verify that the spelling is cor

Generated Cypher:
MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:ACCESSIBLE]->(d:Door) RETURN c.id, d.id
Full Context:
[]

> Finished chain.

Observation: {'query': "MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:HAS_DOOR]->(d:Door) RETURN c.id, d.id, d.separation", 'all_nodes_query': 'MATCH (n)\n    WHERE size(labels(n)) > 0 \n    WITH labels(n)[0] AS label, collect(n) AS nodes\n    RETURN label, nodes[0] AS sampleNode', 'all_nodes': "[{'label': 'Column', 'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjNmt', 'level_id': '79627', 'location': [96.06087148282784, 102.24828131414463, 24.27821522309711], 'id': '2683100'}}, {'label': 'Door', 'sampleNode': {'ifc_guid': '2P0Clh2c952RPyuPFEYgcQ', 'level_id': '24770', 'location': [28.524353627965954, 43.365276144551295, 0.0], 'id': '2642808'}}, {'label': 'Separationline', 'sampleNode': {'ifc_guid': '1yPJv2Vxf8r9twVNmFuZ2v', 'level_id': '2680262', 'location': [39.08435362796594, 27.16527614455126, 11.1, 39.08435362796593, 

Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `GUID` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=1, column=25, offset=24>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 24, 'line': 1, 'column': 25}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:ACCESSIBLE]->(d:Door) RETURN d.id, d.ifc_guid"
Received notification from DBMS server: <GqlStatusObject gql_status='01N50', status_description='warn: label does not exist. The label `CoreComponent` does not exist in database `neo4j`. Verify that the spelling 

Generated Cypher:
MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:ACCESSIBLE]->(d:Door) RETURN d.id, d.ifc_guid
Full Context:
[]

> Finished chain.

Observation: {'query': "MATCH (c:CoreComponent {GUID: '1W6yAWYtbDbPaMjexfopdJ'})-[:HAS_DOOR]->(d:Door) RETURN d.id, d.separation", 'all_nodes_query': 'MATCH (n)\n    WHERE size(labels(n)) > 0 \n    WITH labels(n)[0] AS label, collect(n) AS nodes\n    RETURN label, nodes[0] AS sampleNode', 'all_nodes': "[{'label': 'Column', 'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjNmt', 'level_id': '79627', 'location': [96.06087148282784, 102.24828131414463, 24.27821522309711], 'id': '2683100'}}, {'label': 'Door', 'sampleNode': {'ifc_guid': '2P0Clh2c952RPyuPFEYgcQ', 'level_id': '24770', 'location': [28.524353627965954, 43.365276144551295, 0.0], 'id': '2642808'}}, {'label': 'Separationline', 'sampleNode': {'ifc_guid': '1yPJv2Vxf8r9twVNmFuZ2v', 'level_id': '2680262', 'location': [39.08435362796594, 27.16527614455126, 11.1, 39.08435362796593, 

In [34]:
# Structured AdaptationPlan with full objects per suggestion
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field, field_validator
import json

class Suggestion(BaseModel):
    # Store objects as JSON string to avoid Dict[str, Any] schema issues with OpenAI
    objects_json: str = Field(
        default="[]",
        description="JSON string of full objects referenced by the suggestion (e.g., Space, Door, Wall).",
    )
    ids: Optional[List[str]] = Field(
        default=None,
        description="Optional ids extracted from the objects or context.",
    )
    action: str = Field(description="Concise action proposal")
    reasoning: str = Field(description="Brief reasoning for the action")
    
    @property
    def objects(self) -> List[Dict[str, Any]]:
        """Parse objects_json to get the list of objects."""
        return json.loads(self.objects_json) if self.objects_json else []
    
    @field_validator('objects_json')
    @classmethod
    def validate_objects_json(cls, v: str) -> str:
        """Ensure objects_json is valid JSON."""
        if v:
            json.loads(v)  # Validate JSON
        return v

class AdaptationPlan(BaseModel):
    suggestions: List[Suggestion] = Field(
        description="Exactly three adaptation suggestions",
        min_items=3,
        max_items=3,
        default_factory=list,
    )


last_clause_text = 'Corridor shall have a minimum width of 6 feet in Group E occupancies with an occupant load of 100 or more'

structured_llm_objects = llm.with_structured_output(AdaptationPlan)

agent = initialize_agent(
    tools=[tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=6,
    early_stopping_method="generate",
    handle_parsing_errors=True,
)

agent_prompt_objects = f"""I've got the following issue: 
<start_regulation>{last_clause_text}</start_regulation>
<issue>{issues[-1]}</issue>

Objectives:
1. Use the `graph-cypher-qa` tool to fetch the actual node objects (Space, Door, Wall, Stair, Corridor, etc.) involved in the issue.
2. Preserve the full JSON dictionaries returned by the tool. Do not summarize or reformat them.
3. Produce exactly three adaptation suggestions grounded in those nodes.

Final response format (strict):
Final Answer:
<final_answer>
{{
  "suggestions": [
    {{
      "action": "...",
      "reasoning": "...",
      "objects_json": "[{{\"id\": \"2512768\", \"ifc_guid\": \"1WNDqqV1rAAAY3T7Oc5Qcy\", \"is_room_bounding\": true, \"level_id\": \"24770\", \"location\": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0]}}]",
      "ids": ["..."]
    }},
    ... two more suggestion objects ...
  ]
}}
</final_answer>
Rules:
- Replace the ellipses with real content.
- `objects_json` must be a JSON array string containing the exact node dictionaries you obtained (copy/paste the dicts and ensure the string is valid JSON).
- `ids` must list every `id` present in `objects_json` (empty array if none).
- Make sure 'ids' are consistent with the objects_json. If not all objects corresponding to the ids are present in the objects_json, make further cypher queries to retrieve the missing objects.
- Do not change the design intent of the original building.
- Retrieve the objects to be modified from the graph. Think about what objects can be directly modified using a design tool or its API.
- Provide only suggestions for modifications, do not try to modify the graph.
- Do not output any text before `Final Answer:` or after the JSON object.
"""

agent_answer_objects: str = agent.run(agent_prompt_objects)

# Example wall object format
wall_example = {
    "ifc_guid": "1WNDqqV1rAAAY3T7Oc5Qcy",
    "is_room_bounding": True,
    "level_id": "24770",
    "location": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0],
    "id": "2512768"
}

example_objects_json = json.dumps([wall_example], ensure_ascii=False)

final_plan_objects: AdaptationPlan = structured_llm_objects.invoke(
    f"""Issue: {json.dumps(issues[-1], ensure_ascii=False)}
    AgentAnswer: {agent_answer_objects}
    Produce exactly three suggestions. For each suggestion:
    - Include full JSON objects as a JSON array string in 'objects_json' field.
    - Example format for objects_json (Wall object): '{example_objects_json}'
    - If an 'id' field is present in those objects, extract and set 'ids' to a list of those id values; otherwise leave ids as null.
    - The objects should be the actual node objects from the graph (Space, Door, Wall, etc.) that are relevant to the suggestion."""
)

print(final_plan_objects.model_dump_json(indent=2))




> Entering new AgentExecutor chain...
The user has provided a regulation and an issue related to a 'Corridor' with a specific `core_component_GUID`.
The goal is to fetch the actual node object for this corridor using `graph-cypher-qa` and then provide three adaptation suggestions.

First, I need to extract the `core_component_GUID` from the issue.
`core_component_GUID`: '3O7NDSvVP7jhJX5Gt$LDET'

Then, I will use this GUID to query the graph for the 'Corridor' node.
After retrieving the node, I will formulate three adaptation suggestions based on the issue (corridor width is 5.083 feet, but required is 6 feet).

Action: graph-cypher-qa
Action Input: {"query": "MATCH (c:Corridor {ifc_guid: '3O7NDSvVP7jhJX5Gt$LDET'}) RETURN c"}

> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: <GqlStatusObject gql_status='01N50', status_description='warn: label does not exist. The label `Corridor` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=1, column=10, offset=9>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 9, 'line': 1, 'column': 10}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "MATCH (c:Corridor {ifc_guid: '3O7NDSvVP7jhJX5Gt$LDET'}) RETURN c"


Generated Cypher:
MATCH (c:Corridor {ifc_guid: '3O7NDSvVP7jhJX5Gt$LDET'}) RETURN c
Full Context:
[]

> Finished chain.

Observation: {'query': '{"query": "MATCH (c:Corridor {ifc_guid: \'3O7NDSvVP7jhJX5Gt$LDET\'}) RETURN c"}', 'all_nodes_query': 'MATCH (n)\n    WHERE size(labels(n)) > 0 \n    WITH labels(n)[0] AS label, collect(n) AS nodes\n    RETURN label, nodes[0] AS sampleNode', 'all_nodes': "[{'label': 'Column', 'sampleNode': {'ifc_guid': '2wwkhajRz9dOXaXiRcjNmt', 'level_id': '79627', 'location': [96.06087148282784, 102.24828131414463, 24.27821522309711], 'id': '2683100'}}, {'label': 'Door', 'sampleNode': {'ifc_guid': '2P0Clh2c952RPyuPFEYgcQ', 'level_id': '24770', 'location': [28.524353627965954, 43.365276144551295, 0.0], 'id': '2642808'}}, {'label': 'Separationline', 'sampleNode': {'ifc_guid': '1yPJv2Vxf8r9twVNmFuZ2v', 'level_id': '2680262', 'location': [39.08435362796594, 27.16527614455126, 11.1, 39.08435362796593, 21.56527614455119, 11.1], 'id': '2779188'}}, {'label': 'Slab', 's

In [39]:
agent = initialize_agent(
    tools=[tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=6,
    early_stopping_method="generate",
    handle_parsing_errors=True,
)

agent_prompt_objects = f"""I've got the following issue: 
<start_regulation>{clause_text}</start_regulation>
<issue>{issues[1]}</issue>

Objectives:
1. Use the `graph-cypher-qa` tool to fetch the actual node objects (Space, Door, Wall, Stair, Corridor, etc.) involved in the issue.
2. Preserve the full JSON dictionaries returned by the tool. Do not summarize or reformat them.
3. Produce exactly three adaptation suggestions grounded in those nodes.

Final response format (strict):
Final Answer:
<final_answer>
{{
  "suggestions": [
    {{
      "action": "...",
      "reasoning": "...",
      "objects_json": "[{{\"id\": \"2512768\", \"ifc_guid\": \"1WNDqqV1rAAAY3T7Oc5Qcy\", \"is_room_bounding\": true, \"level_id\": \"24770\", \"location\": [31.679353627965952, 41.56527614455126, 0.0, 31.679353627965913, 15.165276144551276, 0.0]}}]",
      "ids": ["..."]
    }},
    ... two more suggestion objects ...
  ]
}}
</final_answer>
Rules:
- Replace the ellipses with real content.
- `objects_json` must be a JSON array string containing the exact node dictionaries you obtained (copy/paste the dicts and ensure the string is valid JSON).
- `objects_json` must only contain objects that need to be modified, for the proposed action
- `ids` must list every `id` present in `objects_json` (empty array if none).
- Make sure 'ids' are consistent with the objects_json. If not all objects corresponding to the ids are present in the objects_json, make further cypher queries to retrieve the missing objects.
- Try to make realistic suggestions for modifying different object and object types.
- Do not change the design intent of the original building.
- Try to make suggestions that make the situation better, but do not make the situation worse.
- Be creative and think about what is possible.
- Retrieve the objects to be modified from the graph. Think about what objects can be directly modified using a design tool or its API.
- Provide only suggestions for modifications, do not try to modify the graph.
- Return the final answer between the tags <final_answer> and </final_answer>
"""

agent_answer_objects: str = agent.run(agent_prompt_objects)





> Entering new AgentExecutor chain...
The user has provided a regulation clause and an issue related to exit separation in a conference room. I need to provide three adaptation suggestions to resolve this issue.

Here's my plan:
1. **Identify the core component:** The issue is related to a 'Conference Room' with `core_component_GUID`: '1W6yAWYtbDbPaMjexfopdJ'.
2. **Understand the problem:** The `actual_variable_value` (17.40 feet) for 'Required door separation' is less than the `required_variable_value` (24.51 feet). This means the exits/doors serving the conference room are too close together.
3. **Formulate suggestions:** I need to think of ways to increase the separation between the exits/doors of the conference room. This could involve:
    * Moving an existing door.
    * Adding a new door and potentially closing an old one.
    * Modifying the conference room's geometry to increase the diagonal dimension, which would in turn increase the required separation, but this is less li

In [ ]:
{
  "suggestions": [
    {
      "action": "Relocate one of the existing doors to increase the separation distance.",
      "reasoning": "The current separation of 17.40 feet is less than the required 24.51 feet. Moving one of the doors further away from the other will increase this distance. For example, moving door '2682960' to the left or door '2682958' to the right along the wall of the Conference Room '307' would increase the separation.",
      "objects_json": "[{\"ifc_guid\": \"1W6yAWYtbDbPaMjexfopdJ\", \"bound_phy\": [\"2682946\", \"2682934\", \"2682935\", \"2682933\"], \"number\": \"307\", \"bound_vrt\": [], \"level_id\": \"2680262\", \"width\": 4.8000000000000265, \"length\": 14.150000000000007, \"name\": \"Conference Room\", \"location\": [34.086932613103635, 34.967800108295116, 11.1], \"id\": \"2686161\"}, {\"ifc_guid\": \"2wwkhajRz9dOXaXiRcjNox\", \"level_id\": \"2680262\", \"location\": [31.679353627965945, 39.385276144551256, 11.1], \"id\": \"2682960\"}, {\"ifc_guid\": \"2wwkhajRz9dOXaXiRcjNob\", \"level_id\": \"2680262\", \"location\": [36.679353627965945, 39.38527614455124, 11.1], \"id\": \"2682958\"}]",
      "ids": ["2686161", "2682960", "2682958"]
    },
    {
      "action": "Add an additional exit access doorway to the Conference Room '307' in a location that maximizes separation from the existing doors.",
      "reasoning": "Introducing a third exit access doorway, strategically placed, could help meet the required separation distance. This new door should be positioned as far as possible from the existing doors to achieve the necessary separation.",
      "objects_json": "[{\"ifc_guid\": \"1W6yAWYtbDbPaMjexfopdJ\", \"bound_phy\": [\"2682946\", \"2682934\", \"2682935\", \"2682933\"], \"number\": \"307\", \"bound_vrt\": [], \"level_id\": \"2680262\", \"width\": 4.8000000000000265, \"length\": 14.150000000000007, \"name\": \"Conference Room\", \"location\": [34.086932613103635, 34.967800108295116, 11.1], \"id\": \"2686161\"}]",
      "ids": ["2686161"]
    },
    {
      "action": "Reconfigure the internal layout of Conference Room '307' to reduce its maximum overall diagonal dimension.",
      "reasoning": "The required door separation is calculated based on the maximum overall diagonal dimension of the space. By reducing this dimension, for example, by adding a partition wall to create a smaller, compliant area, the required separation distance would decrease, potentially bringing the existing door separation into compliance.",
      "objects_json": "[{\"ifc_guid\": \"1W6yAWYtbDbPaMjexfopdJ\", \"bound_phy\": [\"2682946\", \"2682934\", \"2682935\", \"2682933\"], \"number\": \"307\", \"bound_vrt\": [], \"level_id\": \"2680262\", \"width\": 4.8000000000000265, \"length\": 14.150000000000007, \"name\": \"Conference Room\", \"location\": [34.086932613103635, 34.967800108295116, 11.1], \"id\": \"2686161\"}]",
      "ids": ["2686161"]
    }
  ]
}